# Track data lineage through processing steps

In [ ]:
# A lamindb instance containing bionty schema (skip if you already loaded your instance)

!lamin init --storage bio-lineage --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

In [ ]:
ln.track()

## Track AnnData with cell types, tissues and diseases

### An AnnData with `tissue`, `cell_type`, `disease` in `.obs`

In [ ]:
adata = ln.dev.datasets.anndata_with_obs()

In [ ]:
adata

In [ ]:
adata.obs[["tissue", "cell_type"]].value_counts()

### Register biological metadata and link to the dataset

```{note}

If you don't want auto-populated ontology_ids, use ln.parse(..., from_bionty=False).

Also see: {doc}`docs:guide/parse`
```

In [ ]:
file = ln.File(adata, key="mini_anndata_with_obs.h5ad")

In [ ]:
ln.save(file)

In [ ]:
cell_types = ln.parse(adata.obs.cell_type, lb.CellType.name)
tissues = ln.parse(adata.obs.tissue, lb.Tissue.name)

In [ ]:
ln.add(cell_types)
ln.add(tissues);

In [ ]:
file.cell_types.set(cell_types)
file.tissues.set(tissues)

### Your vocabulary store

In [ ]:
ln.select(lb.CellType).df()

In [ ]:
ln.select(lb.Tissue).df()

## Processing of the dataset

Pull the ingested parent dataset:

In [ ]:
file = ln.select(ln.File, key="mini_anndata_with_obs.h5ad").one()

In [ ]:
adata = file.backed()  # get a cloud-backed AnnData object

In [ ]:
adata

In [ ]:
adata.obs[["cell_type", "disease"]].value_counts()

### Subset dataset to specific cell types and diseases (`.obs`)

In [ ]:
transform = ln.Transform(
    type="pipeline", name="subset_to_T_cells_and_liver_lymphoma", version="0.1.0"
)

In [ ]:
ln.track(transform)

In [ ]:
obs = file.subsetter()

In [ ]:
subset_obs = obs.cell_type.isin(["T cell", "hematopoietic stem cell"]) & (
    obs.disease.isin(["liver lymphoma", "chronic kidney disease"])
)

In [ ]:
adata_subset = file.stream(subset_obs=subset_obs, is_run_input=True)

In [ ]:
adata_subset

In [ ]:
adata_subset.obs[["cell_type", "disease"]].value_counts()

```{tip}

- For `h5ad`, you can provide either `query_obs` OR `query_var` for subsetting (h5py doesn't support indexing for multiple axis).
- For `zarr`, you can provide BOTH. (Note zarr streaming/subset only works with `anndata>=0.9.1`!)

Subset to specific genes (`.var`):

```python
genes = adata.var.index.values[:10]
var = file.subsetter()
adata_subset_var = file.stream(subset_var=var.index.isin(genes))
```

## Add the subset `AnnData` to LaminDB

In [ ]:
file_subset = ln.File(adata_subset, key="subset/mini_anndata_with_obs.h5ad")

In [ ]:
ln.save(file_subset)

Link the subsetted file to cell types:

In [ ]:
cell_types = ln.parse(adata_subset.obs.cell_type, lb.CellType.name)

In [ ]:
file_subset.cell_types.set(cell_types)

## Data lineage

- Which h5ad file is in the `subset` subfolder?
- Which notebook ingested this file?
- By whom?
- And which file is its parent?

Query a subsetted `.h5ad` file containing "hematopoietic stem cell" and "T cell":

In [ ]:
file_subset = (
    ln.select(ln.File, suffix=".h5ad")
    .filter(
        key__startswith="subset",
        cell_types__name__in=["hematopoietic stem cell", "T cell"],
    )
    .first()
)

File:

In [ ]:
file_subset

Which notebook ingested this file?

In [ ]:
file_subset.transform

Who ingested this file?

In [ ]:
file_subset.created_by

Parent files:

In [ ]:
file_subset.run.inputs.values()